In [1]:
from nltk.corpus import reuters
from nltk import FreqDist
import string
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [9]:
def get_stream_details():
    print("Reading the stream details...")
    complete_stream_details_df = pd.read_csv("H:\\TeamStreamz_IW\\code\\data\\card_module_details_content_extracted.csv", encoding="ISO-8859-1")
    if complete_stream_details_df is not None:
        complete_stream_details_dict = {}
        _stream_id_stream_title_dict = {}
        for _, row in complete_stream_details_df.iterrows():
            
            stream_id = str(row["DECKID"])
            stream_title = str(row["DECKNAME"])
            row_content = str(row["HTML_CONTENT"])

            # TODO: add the card title and the module name to the content on which the tags can be generated
            card_title =str(row["CARDTITLE"])
            module_name = str(row["MODULENAME"])
            
            if row_content and "nan" not in row_content:
                # if the stream ID already exists in the dictionary
                if complete_stream_details_dict.get(stream_id):
                    existing_content = complete_stream_details_dict[stream_id]
                    new_content = existing_content + "\n" + row_content.strip()
                    complete_stream_details_dict[stream_id] = new_content
                else:
                    complete_stream_details_dict[stream_id] = row_content.strip()
                    _stream_id_stream_title_dict[stream_id] = stream_title
        
        return complete_stream_details_dict

In [10]:
stream_details_dict = get_stream_details()
stream_details_dict

Reading the stream details...


{'1089': 'Welcome Team! Were really glad you are part of this journey!     In this stream you will find out more about the program and its fantastic benefits.     But first, scroll to the next card to complete a small survey, to help us guide your ...\nThis tool is used to capture the brand survey information.       "On a scale of 1-10, how likely are you to recommend a Castrol brand to a customer?"       You cannot edit this c...\n{"title": "Brand Survey", "primaryField": "field1", "page1": {"header": "{CARD_DESCRIPTION}", "field1": {"type": "Text", "name": "NPS Score", "sequence": 1}, "sequence": 1}, "maxSubmissions": 1}\nUp-to-date       knowledge on Castrol Brand 123 messaging\nInteractive quizzes to let you        track your progress       and growth\nBadges and Competitions that entitle you to        great gifts and rewards!\nThe Castrol Zoom App provides interactive information, competitions and rewards on all things Castrol!     Enjoy the journey!',
 '1095': 'In this Stream you

In [11]:
len(stream_details_dict)

112

In [ ]:
## So we have 112 documents with the contents given by the values

# what is the content length
# largest, average and minimum
# convert case, remove punctuations
# stem the words
# check the frequencies
# perform TF-IDF to get the scores for words for each document
# generate the tags based on the TF-IDF scores